In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/laknath123/PDS_final_project_animalshelter/main/data/merged_df.csv")

In [3]:
df.columns

Index(['Unnamed: 0', 'Animal ID', 'Name_intakes', 'DateTime_intake',
       'MonthYear_intake', 'Found Location', 'Intake Type', 'Intake Condition',
       'Animal Type_intake', 'Sex upon Intake', 'Age upon Intake',
       'Breed_intake', 'Color_intake', 'Name_outcomes', 'DateTime_outcomes',
       'MonthYear_outcomes', 'Date of Birth', 'Outcome Type',
       'Outcome Subtype', 'Animal Type_outcomes', 'Sex upon Outcome',
       'Age upon Outcome', 'Breed_outcomes', 'Color_outcomes', 'age', 'seq',
       'indicator_col'],
      dtype='object')

In [4]:
df = df[(df['indicator_col']=='both')] # get a df of values that contain an outcome so we have a label

In [5]:
df.columns

Index(['Unnamed: 0', 'Animal ID', 'Name_intakes', 'DateTime_intake',
       'MonthYear_intake', 'Found Location', 'Intake Type', 'Intake Condition',
       'Animal Type_intake', 'Sex upon Intake', 'Age upon Intake',
       'Breed_intake', 'Color_intake', 'Name_outcomes', 'DateTime_outcomes',
       'MonthYear_outcomes', 'Date of Birth', 'Outcome Type',
       'Outcome Subtype', 'Animal Type_outcomes', 'Sex upon Outcome',
       'Age upon Outcome', 'Breed_outcomes', 'Color_outcomes', 'age', 'seq',
       'indicator_col'],
      dtype='object')

# Dataset for Building Logistic Regression Classifier

In [6]:
df_unscaled = df[['Animal ID','Intake Type','Sex upon Intake',
     'Intake Condition','Breed_intake','age','Outcome Type']]

In [7]:
df_unscaled.head()

,Animal ID,Intake Type,Sex upon Intake,Intake Condition,Breed_intake,age,Outcome Type
0,A0061001,Public Assist,Neutered Male,Normal,Spinone Italiano Mix,2434.0,Return to Owner
1,A0061002,Public Assist,Neutered Male,Normal,Spinone Italiano Mix,2721.0,Return to Owner
2,A0061003,Stray,Neutered Male,Normal,Spinone Italiano Mix,3804.0,Return to Owner
3,A0477591,Owner Surrender,Neutered Male,Normal,Dachshund,3657.0,Transfer
4,A1340671,Public Assist,Neutered Male,Injured,Shetland Sheepdog,5875.0,Return to Owner


In [8]:
df_unscaled['age'].isnull().sum()

0

In [9]:
# Creating a binary outcome label for the outcome type columns
def label_creation(x):
    if x=='Adoption':
        return(1)
    else:
        return(0)

In [10]:
df_unscaled['Outcome Type']= df_unscaled['Outcome Type'].apply(label_creation)

C:\Users\lakna\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
intake_cat= df_unscaled[['Intake Type']]
sex_cat = df_unscaled[['Sex upon Intake']]
condition_cat = df_unscaled[['Intake Condition']]
breed_cat = df_unscaled[['Breed_intake']]
age_num = df_unscaled[['age']]

In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
encoder = OneHotEncoder()
scaler = StandardScaler()

intake_1hot= encoder.fit_transform(intake_cat)
sex_1hot = encoder.fit_transform(sex_cat)
cond_1hot = encoder.fit_transform(condition_cat)
breed_1hot = encoder.fit_transform(breed_cat)
age_std = scaler.fit_transform(age_num)

In [30]:
from scipy.sparse import hstack
#data = hstack((intake_1hot,sex_1hot,cond_1hot,age_std))
data = hstack((intake_1hot,sex_1hot,age_std))

In [31]:
X,y = data.toarray(),df_unscaled['Outcome Type']

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [39]:
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression(max_iter=500)
log_clf = log_clf.fit(X_train, y_train)

In [40]:
log_clf.predict_proba(X_test)[:,1]

array([0.00139287, 0.4873883 , 0.36617996, ..., 0.52794799, 0.3516417 ,
       0.50981948])

In [41]:
log_clf.predict_proba(X_train) # Predict on the validation set

array([[0.99860836, 0.00139164],
       [0.45464115, 0.54535885],
       [0.56794818, 0.43205182],
       ...,
       [0.96193612, 0.03806388],
       [0.77452332, 0.22547668],
       [0.46127521, 0.53872479]])

In [42]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(log_clf, X_train, y_train, cv=3)

In [20]:
#Each columns of this matrix represents a predicted label
#                            col 1 = predicted not adopted | col 2 = predicted adopted
# row 1 - actual not adopted        34199                  |     15699
# row 2 - actual adopted            21643                  |     20473

In [43]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_train_pred)


array([[28798, 21100],
       [10911, 31205]], dtype=int64)

In [44]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train, y_train_pred)

0.5965968836631297